In [35]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [36]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
import time

In [51]:
service = Service(r'C:\\Users\\Roshan Koirala\\Desktop\\FUN with GPT\\NEPAL PREMIER LEAGUE\\NPL NEW TRY\\geckodriver-v0.35.0-win64 (1)\\geckodriver.exe')

driver = webdriver.Firefox(service=service)

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Firefox()

try:
    url = "https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/match-schedule-fixtures-and-results"
    driver.get(url)
    time.sleep(5)
    
    npl_matches = []
    
    for _ in range(5):  
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        links = driver.find_elements(By.CSS_SELECTOR, "a[href*='nepal-premier-league'][href*='full-scorecard']")
        for link in links:
            href = link.get_attribute('href')
            if href and href not in npl_matches:
                npl_matches.append(href)
    
    for match in npl_matches:
        print(match)
    print(f"\nTotal NPL matches: {len(npl_matches)}")

except Exception as e:
    print(f"Error: {str(e)}")
finally:
    driver.quit()

https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/biratnagar-kings-npl-vs-janakpur-bolts-npl-1st-match-1462596/full-scorecard
https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/kathmandu-gurkhas-npl-vs-chitwan-rhinos-npl-2nd-match-1462640/full-scorecard
https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/janakpur-bolts-npl-vs-karnali-yaks-npl-3rd-match-1462641/full-scorecard
https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/sudur-paschim-royals-npl-vs-biratnagar-kings-npl-4th-match-1462642/full-scorecard
https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/chitwan-rhinos-npl-vs-pokhara-avengers-npl-5th-match-1462643/full-scorecard
https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/karnali-yaks-npl-vs-kathmandu-gurkhas-npl-6th-match-1462644/full-scorecard
https://www.espncricinfo.com/series/nepal-premier-league-2024-25-1462594/biratnagar-kings-npl-vs-lumbini

In [39]:
def extract_team_names(driver):
    team_elements = driver.find_elements(
        By.CSS_SELECTOR, ".ds-text-title-xs.ds-font-bold.ds-capitalize"
    )

    team_names = [team.text.strip() for team in team_elements]

    if len(team_names) >= 2:
        return team_names[0], team_names[1]
    else:
        return None, None

In [50]:
def scrape_match_details(driver, match_url):
    driver.get(match_url)
    wait = WebDriverWait(driver, 10)

    table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ds-table')))

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    match_details = {}

    rows = soup.find_all('tr')
    
    team1, team2 = extract_team_names(driver)
    match_details['Team 1'] = team1
    match_details['Team 2'] = team2
    
    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 2:  # two columns (label and value)
            label = cols[0].text.strip()
            value = cols[1].text.strip()
            
            if 'Toss' in label:
                match_details['Toss'] = value
            elif 'Series' in label:
                match_details['Series'] = value
            elif 'Season' in label:
                match_details['Season'] = value
            elif 'Player Of The Match' in label:
                match_details['Player Of The Match'] = value
            elif 'Match days' in label:
                match_details['Match days'] = value
            elif 'Umpires' in label:
                umpires = [a.text.strip() for a in cols[1].find_all('a')]
                match_details['Umpires'] = umpires
            elif 'TV Umpire' in label:
                match_details['TV Umpire'] = value
            elif 'Points' in label:
                match_details['Points'] = value

    match_info_p = soup.find('p', {'class': 'ds-text-tight-s ds-font-medium ds-truncate ds-text-typo'})
    if match_info_p:
        match_details['Match Info'] = match_info_p.text.strip()
        
    team_divs = soup.find_all('div', {'class': 'ds-text-tight-m ds-font-bold ds-text-typo'})
    if len(team_divs) >= 2:  # Ensure two teams are present
        match_details['Team 1'] = team_divs[0].text.strip()
        match_details['Team 2'] = team_divs[1].text.strip()
        
    target_element = soup.find('div', {'class': 'ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap'})
    if target_element:
        target_value = target_element.find('strong').text.strip()
        match_details['Target'] = target_value
    else:
        match_details['Target'] = 'Not Available'
        
    

    return match_details

In [52]:
matches = []

for url in npl_matches:
    match_details = scrape_match_details(driver, url)
    
    match_info = match_details.get('Match Info', '')
    
    if match_info: 
        match_info_parts = match_info.split(' by ')
        if len(match_info_parts) == 2:
            match_details['Winner_Team'] = match_info_parts[0].split(' ')[0]  
            match_details['won_by_margin'] = match_info_parts[1].strip() 
    
    matches.append(match_details)
    print(match_details)

{'Team 1': 'Biratnagar Kings (NPL)', 'Team 2': 'Janakpur Bolts (NPL)', 'Toss': 'Janakpur Bolts (NPL), elected to field first', 'Series': 'Nepal Premier League', 'Season': '2024/25', 'Player Of The Match': 'Lahiru Milantha', 'Match days': '30 November 2024 - day (20-over match)', 'Umpires': ['Durga Subedi', 'Vinay Kumar'], 'TV Umpire': 'Bismillah Jan Shinwari', 'Points': 'Janakpur Bolts (NPL) 2, Biratnagar Kings (NPL) 0', 'Match Info': 'Janakpur won by 8 wickets (with 27 balls remaining)', 'Target': '127', 'Winner_Team': 'Janakpur', 'won_by_margin': '8 wickets (with 27 balls remaining)'}
{'Team 1': 'Kathmandu Gurkhas (NPL)', 'Team 2': 'Chitwan Rhinos (NPL)', 'Toss': 'Chitwan Rhinos (NPL), elected to field first', 'Series': 'Nepal Premier League', 'Season': '2024/25', 'Player Of The Match': 'Sohail Tanvir', 'Match days': '02 December 2024 - day (20-over match)', 'Umpires': ['Buddhi Pradhan', 'Himal Giri'], 'TV Umpire': 'Durga Subedi', 'Points': 'Chitwan Rhinos (NPL) 2, Kathmandu Gurkhas 

In [49]:
import csv

csv_file_name = "Nepal_Premier_League.csv"

if matches:
    headers = matches[0].keys()  

with open(csv_file_name, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()  
    writer.writerows(matches)  

print(f"Match details have been written to {csv_file_name}")

Match details have been written to Nepal_Premier_League.csv
